In [ ]:
# Googleドライブへのマウント（Colab用コード）
from google.colab import drive
drive.mount('/content/drive')
%cd "/content/drive/My Drive/Colab Notebooks/3D_lecture/"
from google.colab.patches import cv2_imshow # colab内で画像表示関数がうまく動かないので、パッチが提供されている

import cv2
import numpy as np  # PythonのOpenCVでは、画像はnumpyのarrayとして管理される
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline
import glob
from sklearn.preprocessing import normalize

def imshow(img):
  if img.ndim == 3:
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    display(Image.fromarray(img))
  else:
    display(Image.fromarray(img))

In [ ]:
# Read light directions
Lt = np.loadtxt('cat/lights.txt')  # Transpose of L
print('light:', Lt)

# Read images
M = np.array([])
for fname in sorted(glob.glob('cat/*.png')):
    im = cv2.imread(fname, 0)
    if M.size == 0:
        height, width = im.shape
        M = im.reshape((-1, 1))
    else:
        M = np.append(M, im.reshape((-1,1)), axis=1)

# Photometric stereo computation (least-squares)
# M = NL <-> M^T = L^T N^T.
N = np.linalg.lstsq(Lt, M.T)[0].T
N = normalize(N, axis=1)  # normalize to account for diffuse reflectance

In [ ]:
# Visualization
N = np.reshape(N, (height, width, 3))  # Reshape to image coordinates
N[:, :, 0], N[:, :, 2] = N[:, :, 2], N[:, :, 0].copy()  # Swap RGB <-> BGR
N = (N + 1.0) / 2.0  # Rescale
N = N * 255

cv2_imshow(N) # 画像の表示